In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [6]:
import math
import time
import cv2
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
from pydantic import BaseModel

# We want to be able to resize images by scale factor. It is a common practice to preprocess images in computer vision models
scale_factor = 0.5
def resize_image(image, scale_factor):
    # Calculate the new dimensions
    new_width = int(image.shape[1] * scale_factor)
    new_height = int(image.shape[0] * scale_factor)

    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height))

    return resized_image

# Defines constants corresponding to different parts of the body
class GetKeypoint(BaseModel):
    NOSE: int = 0
    LEFT_EYE: int = 1
    RIGHT_EYE: int = 2
    LEFT_EAR: int = 3
    RIGHT_EAR: int = 4
    LEFT_SHOULDER: int = 5
    RIGHT_SHOULDER: int = 6
    LEFT_ELBOW: int = 7
    RIGHT_ELBOW: int = 8
    LEFT_WRIST: int = 9
    RIGHT_WRIST: int = 10
    LEFT_HIP: int = 11
    RIGHT_HIP: int = 12
    LEFT_KNEE: int = 13
    RIGHT_KNEE: int = 14
    LEFT_ANKLE: int = 15
    RIGHT_ANKLE: int = 16

get_keypoint = GetKeypoint()

# Specify the path of the video you want to process
video_path = '../Videos/AnteriorParkinsons.mp4'  # Replace with your video path

# Open video capture
cap = cv2.VideoCapture(video_path)
model = YOLO('yolov8n-pose.pt')  # Ensure you have the correct YOLO model

# Initialize variables for angular and linear velocities (for future calculations)
prev_angle = None
prev_angular_velocity = None
prev_linear_velocity = None
linear_acceleration = None
angular_velocity = None
prev_time = 0
data_list = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Resize the image & get the current frame time
        frame = resize_image(frame, scale_factor)
        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000

        # Get the results from the YOLO model at that current frame with a 0.7 conf
        results = model(frame, conf=0.7)

        # Retrieve keypoints from the result and take their xyn coordinates
        result_keypoint = results[0].keypoints.xyn.cpu().numpy()

        index = 0
        if len(result_keypoint) > 1 and result_keypoint[0][get_keypoint.RIGHT_ANKLE][0] > result_keypoint[1][get_keypoint.RIGHT_ANKLE][0]:
            index = 1
        print('index', index)

        # Extract the keypoints: Nose, Eyes, Ears, Shoulders, Elbows, Wrists, Hips, Knees, and Ankles
        nose = result_keypoint[index][get_keypoint.NOSE]
        left_eye = result_keypoint[index][get_keypoint.LEFT_EYE]
        right_eye = result_keypoint[index][get_keypoint.RIGHT_EYE]
        left_ear = result_keypoint[index][get_keypoint.LEFT_EAR]
        right_ear = result_keypoint[index][get_keypoint.RIGHT_EAR]
        left_shoulder = result_keypoint[index][get_keypoint.LEFT_SHOULDER]
        right_shoulder = result_keypoint[index][get_keypoint.RIGHT_SHOULDER]
        left_elbow = result_keypoint[index][get_keypoint.LEFT_ELBOW]
        right_elbow = result_keypoint[index][get_keypoint.RIGHT_ELBOW]
        left_wrist = result_keypoint[index][get_keypoint.LEFT_WRIST]
        right_wrist = result_keypoint[index][get_keypoint.RIGHT_WRIST]
        left_hip = result_keypoint[index][get_keypoint.LEFT_HIP]
        right_hip = result_keypoint[index][get_keypoint.RIGHT_HIP]
        left_knee = result_keypoint[index][get_keypoint.LEFT_KNEE]
        right_knee = result_keypoint[index][get_keypoint.RIGHT_KNEE]
        left_ankle = result_keypoint[index][get_keypoint.LEFT_ANKLE]
        right_ankle = result_keypoint[index][get_keypoint.RIGHT_ANKLE]

        # Here we calculate angular velocity and other metrics using the right leg (for example purposes)
        vector1 = (right_knee[0] - right_ankle[0], right_knee[1] - right_ankle[1])
        vector2 = (right_hip[0] - right_ankle[0], right_hip[1] - right_ankle[1])

        # Calculate angle between the vectors
        dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]
        magnitude1 = math.sqrt(vector1[0] ** 2 + vector1[1] ** 2)
        magnitude2 = math.sqrt(vector2[0] ** 2 + vector2[1] ** 2)
        angle_rad = math.acos(dot_product / (magnitude1 * magnitude2))
        angle_deg = math.degrees(angle_rad)

        # Get the time difference between frames
        time_diff = current_time - prev_time

        # Calculate angular velocity
        if prev_angle is not None:
            angular_velocity = (angle_deg - prev_angle) / time_diff
            print(f'Angular Velocity: {angular_velocity} deg/s')

        # Calculate linear acceleration if previous angular velocity exists
        if prev_angular_velocity is not None:
            linear_acceleration = (angular_velocity - prev_angular_velocity)

        # Update variables
        prev_angle = angle_deg
        prev_angular_velocity = angular_velocity
        prev_linear_acceleration = linear_acceleration
        prev_time = current_time

        # Append data to the list
        data_list.append({
            'time': current_time,
            'linear_acceleration': linear_acceleration,
            'angular_velocity': angular_velocity
        })

    except Exception as e:
        print(e)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save the DataFrame to a CSV file
df = pd.DataFrame(data_list)
df.to_csv(f'./{Path(video_path).stem}.csv')

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 96.5ms
Speed: 5.0ms preprocess, 96.5ms inference, 9.9ms postprocess per image at shape (1, 3, 384, 640)
index 0

0: 384x640 1 person, 49.1ms
Speed: 0.9ms preprocess, 49.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
index 0
Angular Velocity: 0.0 deg/s

0: 384x640 1 person, 45.5ms
Speed: 1.2ms preprocess, 45.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
index 0
Angular Velocity: 15.13885394544012 deg/s

0: 384x640 1 person, 44.4ms
Speed: 1.0ms preprocess, 44.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
index 0
Angular Velocity: -3.132207644746177 deg/s

0: 384x640 1 person, 47.2ms
Speed: 1.3ms preprocess, 47.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
index 0
Angular Velocity: -2.974342052529229 deg/s

0: 384x640 1 person, 46.4ms
Speed: 0.8ms preprocess, 46.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
index 0
Angular Velocity: 8.639521579051484 deg/

/var/folders/tq/g1l9zgqd2kn51pb90m2vx03w0000gn/T/ipykernel_54259/1546228442.py:107: RuntimeWarning: invalid value encountered in divide
  angle_rad = math.acos(dot_product / (magnitude1 * magnitude2))


0: 384x640 (no detections), 41.0ms
Speed: 0.9ms preprocess, 41.0ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
index 0
index 0 is out of bounds for axis 0 with size 0

0: 384x640 (no detections), 40.4ms
Speed: 1.8ms preprocess, 40.4ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
index 0
index 0 is out of bounds for axis 0 with size 0

0: 384x640 (no detections), 38.5ms
Speed: 0.8ms preprocess, 38.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
index 0
index 0 is out of bounds for axis 0 with size 0

0: 384x640 (no detections), 38.3ms
Speed: 0.9ms preprocess, 38.3ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
index 0
index 0 is out of bounds for axis 0 with size 0

0: 384x640 (no detections), 39.5ms
Speed: 0.9ms preprocess, 39.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)
index 0
index 0 is out of bounds for axis 0 with size 0

0: 384x640 (no detections), 38.3ms
Speed: 0.8ms preproc